This is an example of a Dash app with three interactive plots. Data used in this notebook is available on Kaggle: https://www.kaggle.com/datasets/rush4ratio/video-game-sales-with-ratings?resource=download

Below is a short description of each part of the dash app. There are three graphs:
1. Stacked area plot, showing game releases by year and platform. Dropdowns based on genre and rating 
2. Scatter plot with ratings. Dropdowns based on genre and rating. 
3. Bar plot with global sales by producer and year, filtered by year period and sales volume per year.

Also there are several filters (different dash core components), referred to one or several graphs:
1. Genres dropdown. Multiple choice is available. Allows to select one or more genres and filter the first two graphs. Also option "Select all" is available but it's needed to remove all the other options.
2. Ratings dropdown. Multiple choice isn't available. Allows to choose on of the ratings in the dataset and filter the first two graphs.
3. Years rangeslider. Allows to choose a years interval for data displaying. Affects all the graphs.
4. Sales volume checklist. Multiple choice is available. Allows to select one or more sales bins and filter the sales volumes per year on the third graph.

In [1]:
#import modules
import dash  # use Dash version 1.16.0 or higher for this app to work
from dash import html
from dash import dcc 
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

#work with dataframe, first of all - import dataframe using pandas
df = pd.read_csv('Video_Games_Sales_as_at_22_Dec_2016.csv') #reading stock price dataset 

#intermediate options that will be needed in the future work 
dff = pd.DataFrame.dropna(df, axis=0, how='any')
dff.drop(dff[dff.Year_of_Release < 1995].index, inplace=True)
dff.drop(dff[dff.User_Score == 'tbd'].index, inplace=True)
dff['Year_of_Release'] = dff['Year_of_Release'].apply(lambda x: int(x))
dff['User_Score'] = dff['User_Score'].apply(lambda x: float(x))

game_count = dff.groupby(['Year_of_Release', 'Platform', 'Genre', 'Rating']).agg({'Name': 'count'})
game_count = game_count.reset_index()
game_count.rename(columns={'Name':'Num_of_games'}, inplace=True)

#bins for sales data
bins = [0, 1,5,10,20,50,100, 150,200]

#create dash app layout
app = dash.Dash()   #initialising dash app
app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Games statistics', style = {'textAlign':'center',\
                                            'marginTop':20,'marginBottom':20}),
    html.H4(id = 'H5_genre', children = 'Genres', style = {'textAlign':'left',\
                                            'marginTop':20,'marginBottom':5}),
		dcc.Dropdown(id='genre_dropdown',
            options=[{'label':x, 'value':x} for x in dff.Genre.unique()]
            + [{'label': 'Select all', 'value': 'all_values'}], value = 'all_values',
            multi = True, search_value='', placeholder='Please select genre...',
           persistence=True, persistence_type='memory'),
    html.H5(id = 'H5_genre_info', children = 'If you choose "Select all", please remove other options', style = {'textAlign':'left',\
                  'marginTop':4,'marginBottom':5, 'font-weight': 'normal'}),
    html.Br(),
    html.H4(id = 'H5_rating', children = 'Rating', style = {'textAlign':'left',\
                                            'marginTop':20,'marginBottom':5}),
		dcc.Dropdown(id='rating_dropdown',
            options=[{'label':x, 'value':x} for x in dff.Rating.unique()], 
            multi = False, search_value='',
            value = 'E', placeholder='Please select platform...',
           persistence=True, persistence_type='memory'),
    
    dcc.RangeSlider(id='my-range-slider', min = min(dff['Year_of_Release']),max =  max(dff['Year_of_Release']),
                marks = {int(x):str(int(x)) for x in sorted(dff.Year_of_Release.unique())}, 
                updatemode='drag', value = [1996,2016]),
    html.Br(),
    html.H2(id = 'H2_rel', children = 'Release of games by year and platform', style = {'textAlign':'center',\
                                            'marginTop':20,'marginBottom':0}),
    html.H5(id='output_container', children= [''], style = {'textAlign':'center'}),

    dcc.Graph(id = 'area_plot'),
		html.Br(),
    html.H2(id = 'H2_scor', children = 'Scores of games by critics and users', style = {'textAlign':'center',\
                                            'marginTop':20,'marginBottom':0}),
    dcc.Graph(id = 'scatter_plot'),
    html.Br(),
    html.H2(id = 'H2_bar', children = 'Global sales of publishers', style = {'textAlign':'center',\
                                            'marginTop':20,'marginBottom':0}),
    dcc.Graph(id = 'bar_plot'),
    html.Br(),
    dcc.Checklist(id = 'checklist', options= [{'label':f'from {bins[i]} to {bins[i+1]} mln $', 'value': [bins[i], bins[i+1]]} for i in range(0, len(bins) -1)],
                  value=[[10,20]], style =  {'textAlign':'center', 'border':'2px green solid'})])

#callback for connecting dash components with figures        
@app.callback(
    [Output(component_id='area_plot', component_property='figure'),
    Output(component_id='scatter_plot', component_property='figure'),
    Output(component_id='output_container', component_property='children'),
    Output(component_id='bar_plot', component_property='figure')
    ],
    [Input(component_id='genre_dropdown', component_property='value'),
    Input(component_id='rating_dropdown', component_property='value'),
    Input(component_id = 'my-range-slider',component_property='value'),
    Input(component_id = 'checklist',component_property='value')]
)
#Creating charts:  
#1. Stacked area plot, showing game releases by year and platform. Dropdowns based on genre and rating 
#2. Scatter plot with ratings. Dropdowns based on genre and rating. 
#3. Bar plot with global sales by producer and year, filtered by year period and sales volume per year.

#function for the callback
def draw_plot(dropdown_genre, dropdown_rating, years, sums):
  #Preparing datasets for the graphs
  if dropdown_genre == 'all_values' or dropdown_genre == ['all_values']:
       dropdown_genre = list(game_count.Genre.unique())
  if type(dropdown_genre) == str:
    dropdown_genre = [dropdown_genre]

  df1 = game_count[(game_count.Genre.isin(dropdown_genre)) & (game_count.Rating.eq(dropdown_rating))]
  df1 = df1[df1['Year_of_Release'].between(years[0], years[1])]

  df2 = dff[(dff.Genre.isin(dropdown_genre)) & (dff.Rating.eq(dropdown_rating))]
  df2 = df2[df2['Year_of_Release'].between(years[0], years[1])]
  
  df3 = dff.groupby(['Publisher', 'Year_of_Release']).agg({'Global_Sales':'sum'}).reset_index()
  df3 = df3[df3['Year_of_Release'].between(years[0], years[1])]
  sums_trans = [item for sublist in sums for item in sublist]
  df3 = df3[df3['Global_Sales'].between(min(sums_trans), max(sums_trans))]

  #Area plot
  fig1 = px.area(df1, x="Year_of_Release", y="Num_of_games", 
  color = 'Platform', line_group='Genre')    
  
  #Scatter plot
  fig2 = px.scatter(df2, x= 'Critic_Score', y = 'User_Score', color = 'Genre', hover_data= ['Genre', 'Name']) 
  container = "Total number of games (on all platforms): {}".format(str(df1.Num_of_games.sum()))
  
  #Bar plot
  fig3 = px.bar(df3,  x = 'Publisher', y = 'Global_Sales',
                color = 'Year_of_Release', color_continuous_scale=px.colors.sequential.Viridis)  
  return fig1, fig2, container, fig3

#launch of the app
if __name__ == '__main__': 
    app.run_server()

    

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/home/nyak/.local/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/tmp/ipykernel_46378/2155161080.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['Year_of_Release'] = dff['Year_of_Release'].apply(lambda x: int(x))
/tmp/ipykernel_46378/2155161080.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u